In [1]:
import os
os.chdir("../../../")

In [2]:
import re

# Parsing Answers from LLaMa-2

In [3]:
import pandas as pd

In [4]:
INPUT_PATH = r"D:\Studying\UoR\1. Data Mining\Final_Project\data\generated\answer_russia.csv"

In [5]:
df = pd.read_csv(INPUT_PATH)
df.sample(3)

,date_publish,maintext,title,source_type,source_name,country,text,generated_text
1232,2022-08-03 12:07:01,German Chancellor Olaf Scholz on Wednesday rai...,Scholz Opens Door To Extend Nuclear as Russia ...,local,moscow_times,russia,"As a neutral news analyst, assess the sentimen...","###Human:\nAs a neutral news analyst, assess t..."
1538,2022-03-11 05:34:19,The UN Security Council will hold an emergency...,UN Security Council to Meet Friday on Biologic...,local,moscow_times,russia,"As a neutral news analyst, assess the sentimen...","###Human:\nAs a neutral news analyst, assess t..."
110,2023-03-16 16:18:32,Russia's Supreme Court on Thursday granted a m...,Russian Supreme Court Deems Freedom of Russia ...,local,moscow_times,russia,"As a neutral news analyst, assess the sentimen...","###Human:\nAs a neutral news analyst, assess t..."


In [6]:
df.shape

(4000, 8)

How many has answers from LLaMa-2

In [7]:
df.generated_text[df.generated_text != ""].count()

4000

# Trimming to only Assistance Portion

In [8]:
def trim(text):
    pattern = "###Assistant:\n"
    matches = [match for match in re.finditer(pattern, text)]
    
    if matches is None:
        return ""
    
    try:
        return text[matches[0].end():].strip()
    except Exception:
        return None

In [9]:
print(trim(df.generated_text.iloc[0]))

1. Sentiment: Positive
    * Score: 0.8
    * Reason: The article conveys a positive sentiment, focusing on the progress made by Ukraine in retaking villages and the morale-boosting statements from President Zelensky.
2. Stance: Pro-Ukraine
    * Score: 0.9
    * Reason: The article takes a clear stance in favor of Ukraine, highlighting the courage and determination of its armed forces and their successful counter-offensive against Russian forces.


In [10]:
df.generated_text = df.generated_text.map(trim)

In [11]:
from src.llama2.parse_csv import ParseCSV

parser = ParseCSV()
df = parser(df)
df.sample(3)

,date_publish,maintext,title,source_type,source_name,country,text,generated_text,sentiment_class,stance_class,sentiment_score,stance_score,sentiment_reason,stance_reason
3726,2020-05-08 09:05:51,The bombastic military parade through Moscow's...,Coronavirus Deals 'Powerful Blow' to Putin's G...,local,moscow_times,russia,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Negative\n * Score: -0.7\n ...,negative,impartial,-0.7,0.0,The article presents a negative view of the im...,The article is reporting on the situation in R...
1000,2023-11-10 19:11:26,Ukrainian and Russian officials on Friday repo...,A teenager taken from occupied Mariupol to Rus...,international,ap,russia,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Positive\n * Score: 0.9\n ...,positive,impartial,0.9,0.1,"The article conveys a positive sentiment, as i...","The article presents both sides of the story, ..."
399,2023-10-12 09:42:10,Deputy Prime Minister Alexander Novak said on ...,Novak: Russia's oil exports cuts include oil p...,international,reuters,russia,"As a neutral news analyst, assess the sentimen...",1. Sentiment: Negative\n * Score: -0.7\n ...,negative,impartial,-0.7,0.0,"The article presents a negative sentiment, as ...",The article maintains an impartial stance by o...


In [12]:
df[(df.country == "canada") & (df.generated_text != "")].source_type.value_counts()

Series([], Name: source_type, dtype: int64)

In [13]:
df.drop(columns=["text", "generated_text"], inplace=True)
df.columns

Index(['date_publish', 'maintext', 'title', 'source_type', 'source_name',
       'country', 'sentiment_class', 'stance_class', 'sentiment_score',
       'stance_score', 'sentiment_reason', 'stance_reason'],
      dtype='object')

In [14]:
df.to_csv(r"D:\Studying\UoR\1. Data Mining\Final_Project\data\parsed\russia.csv", index=False)